\[<< [Building command line interface](./08_argparse/09_argparse.ipynb) | [Index](./00_index.ipynb) | [Multithreading and Multiprocessing](./10_concurrency.ipynb) >>\]


# Unit Testing

In [1]:
%load_ext save_and_exec_magic

In [2]:
%%save_and_run_magic unittest
import unittest

class ExampleTest(unittest.TestCase):
    def test_something(self):
        val = 1
        self.assertEqual(val, 5)

In [5]:
%pip install pytest

   ---------------------------------------- 0.0/325.1 kB ? eta -:--:--
   --- ----------------------------------- 30.7/325.1 kB 660.6 kB/s eta 0:00:01
   --------------- ------------------------ 122.9/325.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 276.5/325.1 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 325.1/325.1 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/325.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/325.1 kB ? eta -:--:--
   -------- ------------------------------ 71.7/325.1 kB 787.7 kB/s eta 0:00:01
   --------------------------- ------------ 225.3/325.1 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 325.1/325.1 kB 2.0 MB/s eta 0:00:00


Note: you may need to restart the kernel to use updated packages.


Similar thing in pytest:

In [6]:
%%save_and_run_magic pytest
def test_something():
    val = 1
    assert val == 5

1. Pytest provides clear breakdowns of test results, including the value of `val`, even though it uses the Python assert statement. This allows for easy identification of test failures and their specific details.

2. Pytest does not require setting up a class for test cases, although you have the option to use classes if needed. This makes writing tests more straightforward and less verbose compared to some other testing frameworks.

3. Unlike other testing frameworks that have numerous `self.assert*` functions for different assertions, pytest simplifies the process by using the familiar Python assert statement for making assertions in tests.

4. Pytest is versatile and can run not only pytest-specific tests but also unittest tests and tests written for the old nose package. This makes it compatible with various testing frameworks, providing flexibility to developers.

## Practice TDD

In [7]:
%%save_and_run_magic pytest
def add(num1, num2):
    pass


def test_add():
    assert add(1, 2) == 3

In [8]:
%%save_and_run_magic pytest
def add(num1, num2):
    return num1 + num2


def test_add():
    assert add(1, 2) == 3

## Use `pytest.approx` for float operation

In [9]:
%%save_and_run_magic pytest
def add(num1, num2):
    return num1 + num2


def test_add():
    assert add(1, 2) == 3

def test_add_float():
    assert add(0.1, 0.2) == 0.3

In [10]:
%%save_and_run_magic pytest
def add(num1, num2):
    return num1 + num2

import pytest

def test_add():
    assert add(1, 2) == 3

def test_add_float():
    assert add(0.1, 0.2) == pytest.approx(0.3)

## Use `pytest.mark.parametrize` for handling unit test which have similar parameters

In [11]:
%%save_and_run_magic pytest
def add(num1, num2):
    return num1 + num2


import pytest


@pytest.mark.parametrize(
    ('input_arg', 'expected'),
    (
        ((1, 2), 3),
        ((0.1, 0.2), 0.3),
    ),
)
def test_add(input_arg, expected):
    assert add(*input_arg) == pytest.approx(expected)


In [12]:
%%save_and_run_magic pytest -v
def add(num1, num2):
    return num1 + num2


import pytest


@pytest.mark.parametrize(
    ('input_arg', 'expected'),
    (
        ((1, 2), 3),
        ((0.1, 0.2), 0.3),
    ),
)
def test_add(input_arg, expected):
    assert add(*input_arg) == pytest.approx(expected)


By default any print statement inside test function will not show up. If your want them use `-s` flag.

In [13]:
%%save_and_run_magic pytest -vs
def add(num1, num2):
    return num1 + num2


import pytest


@pytest.mark.parametrize(
    ('input_arg', 'expected'),
    (
        ((1, 2), 3),
        ((0.1, 0.2), 0.3),
    ),
)
def test_add(input_arg, expected):
    print(f"\n{input_arg = }, {expected = }")
    assert add(*input_arg) == pytest.approx(expected)


Use `pytest.raises` to check if an expected exception is raised

In [14]:
%%save_and_run_magic pytest -v
import pytest

def test_raises():
    with pytest.raises(ZeroDivisionError):
        1 / 0

## pytest fixtures

- Pytest fixtures are functions used in tests for setup, teardown, and handling resources.
- They are recognized by names in test functions and automatically invoked by pytest.
- Fixtures centralize setup logic, making tests modular, reusable, and maintainable.
- Pytest provides built-in fixtures and allows custom fixtures to enhance testing capabilities.
- Fixtures support parameterization and different scopes for precise control over their usage.

### fixture as data

In [15]:
%%save_and_run_magic pytest -v
class Calculator:
    def add(self, a, b):
        return a + b

    def subtract(self, a, b):
        return a - b

def test_addition():
    calc = Calculator()
    result = calc.add(5, 10)
    assert result == 15

def test_subtraction():
    calc = Calculator()
    result = calc.subtract(20, 8)
    assert result == 12

In [16]:
%%save_and_run_magic pytest -v
class Calculator:
    def add(self, a, b):
        return a + b

    def subtract(self, a, b):
        return a - b

import pytest

@pytest.fixture
def calc():
    return Calculator()

def test_addition(calc):
    result = calc.add(5, 10)
    assert result == 15

def test_subtraction(calc):
    result = calc.subtract(20, 8)
    assert result == 12

### fixture as test state

In [19]:
%%save_and_run_magic pytest -vs
import pytest

@pytest.fixture
def setup_and_teardown_example():
    print("Setup - Before the test")
    yield
    print("Teardown - After the test")

def test_add(setup_and_teardown_example):
    print("Running test_add")
    assert 1 + 1 == 2

@pytest.mark.usefixtures('setup_and_teardown_example')
def test_sub():
    print("Running test_sub")
    assert 1 - 1 == 0

### run fixture for everything

In [20]:
%%save_and_run_magic pytest -vs
import pytest

@pytest.fixture(autouse=True)
def setup_and_teardown_example():
    print("Setup - Before the test")
    yield
    print("Teardown - After the test")

def test_add():
    print("Running test_add")
    assert 1 + 1 == 2

def test_sub():
    print("Running test_sub")
    assert 1 - 1 == 0

By default the fixture is scope in function. So for each function the fixture is invoked. But we can set it to some specific scope

In [21]:
%%save_and_run_magic pytest -vs
import pytest

@pytest.fixture(autouse=True, scope='session')
def setup_and_teardown_example():
    print("Setup - Before the test")
    yield
    print("Teardown - After the test")

def test_add():
    print("Running test_add")
    assert 1 + 1 == 2

def test_sub():
    print("Running test_sub")
    assert 1 - 1 == 0

\[<< [Building command line interface](./08_argparse/09_argparse.ipynb) | [Index](./00_index.ipynb) | [Multithreading and Multiprocessing](./10_concurrency.ipynb) >>\]
